In [1]:
from torchvision.models import vgg16
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import VOCDetection
import torch.optim as optim
import PIL.Image as Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont

In [2]:
import imageio
import math

In [3]:
from torchvision.transforms import Resize

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
FONT = ImageFont.truetype('arial.ttf', 24)

In [7]:
#数据处理

In [8]:
train_tramforms=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [9]:
train=VOCDetection('C:/数据/目标检测voc2007/',year='2007',image_set='train',download=False,transform=train_tramforms)

In [10]:
train

Dataset VOCDetection
    Number of datapoints: 2501
    Root location: C:/数据/目标检测voc2007/
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [11]:
train[1][1]['annotation']['object']

[{'name': 'person',
  'pose': 'Left',
  'truncated': '0',
  'difficult': '0',
  'bndbox': {'xmin': '185', 'ymin': '62', 'xmax': '279', 'ymax': '199'}},
 {'name': 'horse',
  'pose': 'Left',
  'truncated': '0',
  'difficult': '0',
  'bndbox': {'xmin': '90', 'ymin': '78', 'xmax': '403', 'ymax': '336'}}]

In [12]:
def get_coo_image(train):
    data_set=[]
    images=[]
    for i in range(len(train)):
        i+=1
        data=[]
        for j in range(len(train[i][1]['annotation']['object'])):
        
            xmin=int(train[i][1]['annotation']['object'][j]['bndbox']['xmin'])
            ymin=int(train[i][1]['annotation']['object'][j]['bndbox']['ymin'])
            xmax=int(train[i][1]['annotation']['object'][j]['bndbox']['xmax'])
            ymax=int(train[i][1]['annotation']['object'][j]['bndbox']['ymax'])
            name=train[i][1]['annotation']['object'][j]['name']
            data.append([xmin,ymin,xmax,ymax,name])
            
        image=train[i][0]
        data_set.append(data)
        
        images.append(image)
        if(i==1):
            break
    return data_set,images

In [13]:
dataset,images=get_coo_image(train)

In [14]:
name=[]
for i in range(len(dataset)):
    for j in range(len(dataset[i])):
        
        name.append(dataset[i][j][-1])

In [15]:
name=list(set(name))

In [16]:
for i in range(len(dataset)):
    for j in range(len(dataset[i])):
        dataset[i][j][-1]=name.index(dataset[i][j][-1])+1

In [17]:
#vgg16预训练

In [18]:
vgg16=vgg16(pretrained=True)

In [19]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [20]:
vgg16=vgg16.features[0:24]

In [21]:
#gt_box=[x,y,x,y,class]

In [22]:
class get_base_anchors(nn.Module):
    def __init__(self,width,height,change_much,ratio=[0.5,1,2],scale=2**torch.arange(3,6)):
        super(get_base_anchors,self).__init__()
        
        
        self.width=width
        self.height=height
        self.change_much=change_much
        
        self.ratio=ratio
        self.scale=scale
        
    def get_box(self,ws,hs,center_x,center_y):#返回左上，右下
        
        ws=ws.unsqueeze(1)
        hs=hs.unsqueeze(1)
        
        left=center_x-(ws-1)/2
        left=left.reshape(-1,1)
        top=center_y-(hs-1)/2
        top=top.T
        top=top.reshape(-1,1)
        
        
        
        right=center_x+(ws-1)/2
        right=right.T
        right=right.reshape(-1,1)
        bottom=center_y+(hs-1)/2
        bottom=bottom.T
        bottom=bottom.reshape(-1,1)
        
        box=torch.hstack((left,top,right,bottom))
      
        return box
        
        
        
    def change_on_ratio(self):#产生3个基于ratio的box
        ratio=torch.tensor(self.ratio)
        base=torch.tensor([0,0,15,15])
        
        size=(base[2]-base[0]+1)*(base[3]-base[1]+1)
        
        
        ws=base[2]-base[0]+1
        hs=base[3]-base[1]+1
        
        ws=ws*ratio
        hs=size/ws
        center_x=(base[0]+base[2])/2
        center_y=(base[1]+base[3])/2
        
        
        box=self.get_box(ws,hs,center_x,center_y)
        #print(box)
        
        
        return box
    
    def change_on_scale(self,box):
        
        scale=torch.tensor(self.scale)
        scale=scale.unsqueeze(1)
        
        center_x=(box[0][0]+box[0][2])/2
        
        center_y=(box[0][1]+box[0][3])/2

        ws=box[:,2]-box[:,0]+1#[3,]
        hs=box[:,3]-box[:,1]+1
        
        ws=scale*ws
        ws=ws.reshape(-1,1)
        hs=scale*hs
        hs=hs.reshape(-1,1)
        
        box=self.get_box(ws,hs,center_x,center_y)
        
        return box
    
    def get_all_anchors(self,box):
        
        
        width=self.width
        height=self.height
        
        
        
        change_much=self.change_much
        ws=box[:,2]-box[:,0]+1
        hs=box[:,3]-box[:,1]+1
        
        xs,ys=torch.meshgrid(torch.arange(width),torch.arange(height))
        xs=(xs.reshape(-1,1).squeeze()+1)*16
        ys=(ys.reshape(-1,1).squeeze()+1)*16
        
        all_anchors=self.get_box(ws,hs,xs,ys)#[9*n,4]
 
        return all_anchors
   
  
      
    def forward(self):
        
        box=self.change_on_ratio()
        box=self.change_on_scale(box)
        
        box=self.get_all_anchors(box)
        return box

In [23]:
class get_lots_on_base_anchors(nn.Module):
    def __init__(self,rpn_cls_score,gt_box,anchors,change_much):
        super(get_lots_on_base_anchors,self).__init__()
        
        self.rpn_cls_score=rpn_cls_score
        self.gt_box=gt_box
        self.anchors=anchors
        self.change_much=change_much
        #self.height,self.width=rpn_cls_score.size()[1:3]

        
    def crop_all_anchors(self,box):#裁剪出去的anchors
        
        box=box[
            (box[:,0]>=0)&(box[:,1]>=0)&(box[:,2]<=self.width*self.change_much)&(box[:,3]<=self.height.change_much)
        ]
        
        return box
        
        
        
    def get_iou(self,boxes,gt_anchors):
        
        N=boxes.size()[0]
        K=gt_anchors.size()[0]
        
        all_iou=torch.zeros((N,K))
        
        for n in range(N):
            size_n=(boxes[n,2]-boxes[n,0]+1)*(boxes[n,3]-boxes[n,1]+1)
            
            for k in range(K):
                
                left=max(boxes[n,0],gt_anchors[k,0])
                right=min(boxes[n,2],gt_anchors[k,2])
                w=right-left+1
                
                if(w>=0):
                    
                    top=max(boxes[n,1],gt_anchors[k,1])
                    bottom=min(boxes[n,3],gt_anchors[k,3])
                    h=bottom-top+1
                    
                    if(h>=0):
                        
                        size=w*h
                        iou=size/((gt_anchors[k,2]-gt_anchors[k,0]+1)*(gt_anchors[k,3]-gt_anchors[k,1]+1)+size_n-size)
                        
                        all_iou[n,k]=iou
        
        return all_iou
    def get_targets(self,anchors,gt_box):
        
        ws=anchors[:,2]-anchors[:,0]+1
        hs=anchors[:,3]-anchors[:,1]+1
        
        centers_x=(anchors[:,0]+anchors[:,2])/2
        centers_y=(anchors[:,3]+anchors[:,1])/2
        
        gt_ws=(gt_box[:,2]-gt_box[:,0])/2
        gt_hs=(gt_box[:,3]-gt_box[:,1])/2
        
        gt_center_x=(gt_box[:,0]+gt_box[:,2])/2
        gt_center_y=(gt_box[:,1]+gt_box[:,3])/2
        #targets_dw = np.log(gt_widths / ex_widths)

        d_x=(gt_center_x-centers_x)/ws
        d_x=d_x.reshape(-1,1)
        d_y=(gt_center_y-centers_y)/hs
        d_y=d_y.reshape(-1,1)     
        d_w=torch.log(gt_ws/ws)
        d_w=d_w.reshape(-1,1)
        d_h=torch.log(gt_hs/hs)
        d_h=d_h.reshape(-1,1)
        
        
        targets=torch.hstack((d_x,d_y,d_w,d_h))
        
        
        return targets
    
        
    # def _unmap(data, count, inds, fill=0):
    def unmap(self,data,count,inds,fill=0):
        
        if len(data.size())==1:
            ret=torch.empty((count,),dtype=torch.float32)
            ret=ret.fill_(fill)
            ret[inds]=data
            
            return ret
        
        else:
            
            ret=torch.empty((count,)+data.size()[1:],dtype=torch.float32)
            ret=ret.fill_(fill)
            ret[inds]=data
            
            return ret
        
    def get_lots_im(self):
        gt_box=self.gt_box
        anchors=self.anchors
        rpn_cls_score=self.rpn_cls_score
        height,width=rpn_cls_score.size()[2:4]
        
        h=height
        w=width
        change_much=self.change_much
        
        N=anchors.size()[0]
        K=gt_box.size()[0]
        
        labels=-1*torch.ones(N)
        box_ind_weight=torch.zeros(N,4,dtype=torch.float64)
        box_out_weight=torch.zeros(N,4,dtype=torch.float64)
        #对anchors进行第一次筛选
        ind=torch.where((anchors[:,0]>=0)
                        &(anchors[:,1]>=0)
                        &(anchors[:,2]<=width*change_much)
                        &(anchors[:,3]<=height*change_much))
        
        anchors1=anchors[ind]
        labels1=labels[ind]
        
        #对anchors进行第二次筛选
        all_iou=self.get_iou(anchors1,gt_box)
        K_anchors_index=torch.argmax(all_iou,axis=0)
        K_iou=all_iou[K_anchors_index,torch.arange(K)]
        
        N_anchors_index=torch.argmax(all_iou,axis=1)
        N_iou=all_iou[torch.arange(len(labels1)),N_anchors_index]
        
        
        labels1[torch.where(N_iou>0.5)]=1
        labels1[torch.where(N_iou<0.2)]=0
        labels1[K_anchors_index]=1
        targets=self.get_targets(anchors1,gt_box[N_anchors_index,:])
        labels=self.unmap(labels1,N,ind,fill=-1)
        targets=self.unmap(targets,N,ind,fill=0)
        
        box_ind_weight[torch.where(labels==1)[0],:]=torch.tensor([1,1,1,1],dtype=torch.float64)
        p_weight=torch.empty((4,),dtype=torch.float64)
        n_weight=torch.empty((4,),dtype=torch.float64)
        p_weight=p_weight.fill_(-1/torch.sum(labels==1,dtype=torch.float64))
        n_weight=n_weight.fill_(2/torch.sum(labels==0,dtype=torch.float64))
        
        box_out_weight[torch.where(labels==1)[0],:]=p_weight
        box_out_weight[torch.where(labels==0)[0],:]=n_weight
        #print(len(labels))
        labels=labels.reshape((1,h,w,9)).permute(0,3,1,2)
        labels=labels.reshape(-1)
        
        targets=targets.reshape((1,9*4,h,w))
        box_ind_weight=box_ind_weight.reshape((1,4*9,h,w))
        box_out_weight=box_out_weight.reshape((1,4*9,h,w))
        
        return labels,targets,box_ind_weight,box_out_weight
     

In [24]:
class get_proposal(nn.Module):
    def __init__(self,rpn_cls,rpn_bbox,anchors,meet,pre_num,after_num,imgae):
        super(get_proposal,self).__init__()
        self.meet=meet
        self.rpn_cls=rpn_cls
        self.rpn_bbox=rpn_bbox
        self.anchors=anchors
        self.pre_num=pre_num
        self.after_num=after_num
        self.imgae=imgae
        
    def get_targets(self,anchors,gt_box):
        #print(gt_box)
       
      
        ws=anchors[:,2]-anchors[:,0]+1
        hs=anchors[:,3]-anchors[:,1]+1
        
        centers_x=(anchors[:,0]+anchors[:,2])/2
        centers_y=(anchors[:,3]+anchors[:,1])/2
        
        gt_ws=(gt_box[:,2]-gt_box[:,0])/2
        gt_hs=(gt_box[:,3]-gt_box[:,1])/2
       
        gt_center_x=(gt_box[:,0]+gt_box[:,2])/2
        gt_center_y=(gt_box[:,1]+gt_box[:,3])/2
        #targets_dw = np.log(gt_widths / ex_widths)

        d_x=(gt_center_x-centers_x)/ws
        d_x=d_x.reshape(-1,1)
        d_y=(gt_center_y-centers_y)/hs
        d_y=d_y.reshape(-1,1)     
        d_w=torch.log(gt_ws/ws)
        d_w=d_w.reshape(-1,1)
        d_h=torch.log(gt_hs/hs)
        d_h=d_h.reshape(-1,1)
       
        targets=torch.hstack((d_x,d_y,d_w,d_h))
        
        return targets
        
   
        
    def bbox_and_targets(self,anchors,rpn_bbox_pred):
        
        ws=anchors[:,2]-anchors[:,0]+1
        hs=anchors[:,3]-anchors[:,1]+1
        center_x=(anchors[:,0]+anchors[:,2])/2
        center_y=(anchors[:,1]+anchors[:,3])/2
        
        pred_x=rpn_bbox_pred[:,0]*ws+center_x
        pred_y=rpn_bbox_pred[:,1]*hs+center_y
        pred_w=torch.exp(rpn_bbox_pred[:,2])*ws
        pred_h=torch.exp(rpn_bbox_pred[:,3])*hs
        
        p_box_0=torch.round(pred_x-(pred_w-1)/2).reshape((-1,1))
        p_box_1=torch.round(pred_y-(pred_h-1)/2).reshape((-1,1))
        p_box_2=torch.round(pred_x+(pred_w-1)/2).reshape((-1,1))
        p_box_3=torch.round(pred_y+(pred_h-1)/2).reshape((-1,1))
        
        p_bbox=torch.hstack((p_box_0,p_box_1,p_box_2,p_box_3))
        return p_bbox
        
    def crop_box(self,box):
        
        
        height,width=self.imgae.size()[2:4]
        box[box[:,0]<0]=0
        box[box[:,1]<0]=0
        box[box[:,2]>width]=width
        box[box[:,3]>height]=height
        
        return box
    def nms(self,proposals,scores):
        
        meet=self.meet
        left=proposals[:,0]
        top=proposals[:,1]
      
        right=proposals[:,2]
        bottom=proposals[:,3]
        
        keep=[]
        area=(right-left+1)*(bottom-top+1)
        index=torch.arange(scores.size()[0])
        
        while index.size()[0]>0:
            
            i=index[0]
            keep.append(i)
           
            yy1=torch.maximum(top[i],top[index[1:]])
            xx1=torch.maximum(left[i],left[index[1:]])
            
            xx2=torch.minimum(right[i],right[index[1:]])
            yy2=torch.minimum(bottom[i],bottom[index[1:]])
            
            ins=(xx2-xx1+1)*(yy2-yy1+1)
            area1=(right[index[1:]]-left[index[1:]]+1)*(bottom[index[1:]]-top[index[1:]]+1)
            area2=(1+bottom[i]-top[i])*(1+right[i]-left[i])
            
            iou=ins/(area1+area2-ins)
            where=torch.where(iou<meet)[0]
           
            index=index[where+1]
            
        keep=torch.tensor(keep)
        scores=scores[keep]
        proposals=proposals[keep]
        return scores,proposals,keep
            
            
        
    def get_rois(self):
        
        
        anchors=self.anchors
        rpn_cls=self.rpn_cls
        rpn_bbox=self.rpn_bbox
        
        rpn_scores=rpn_cls[:,9:,:,:]
        rpn_bbox_pred=rpn_bbox.reshape((-1,4))
        rpn_scores=rpn_scores.reshape((-1,))
        proposals=self.bbox_and_targets(anchors,rpn_bbox_pred)
        proposals=self.crop_box(proposals)
        #print(len(proposals))

        
        rpn_scores,indexs=rpn_scores.sort(descending = True)
        indexs=indexs.squeeze()
        proposals=proposals[indexs,:]
       # print(proposals.size())

        #预筛选
        indexs=indexs[:self.pre_num]
        proposals=proposals[:self.pre_num]
        rpn_scores=rpn_scores[:self.pre_num]
        scores,proposals,keep=self.nms(proposals,rpn_scores)
       # print(proposals)
        #后筛选
        scores=scores[:self.after_num]
        proposals=proposals[:self.after_num]
        
        return proposals,scores,keep
    
    def get_iou(self,boxes,gt_anchors):
        
        N=boxes.size()[0]
        K=gt_anchors.size()[0]
        
        all_iou=torch.zeros((N,K))
        
        for n in range(N):
            size_n=(boxes[n,2]-boxes[n,0]+1)*(boxes[n,3]-boxes[n,1]+1)
            
            for k in range(K):
                
                left=max(boxes[n,0],gt_anchors[k,0])
                right=min(boxes[n,2],gt_anchors[k,2])
                w=right-left+1
                
                if(w>=0):
                    
                    top=max(boxes[n,1],gt_anchors[k,1])
                    bottom=min(boxes[n,3],gt_anchors[k,3])
                    h=bottom-top+1
                    
                    if(h>=0):
                        
                        size=w*h
                        iou=size/((gt_anchors[k,2]-gt_anchors[k,0]+1)*(gt_anchors[k,3]-gt_anchors[k,1]+1)+size_n-size)
                        
                        all_iou[n,k]=iou
        
        return all_iou
#bbox_target,ind_weight=self.get_final_targets(labels,bbox_target)
    def get_final_targets(self,labels,box_target):
        bbox_targets=torch.zeros((labels.size()[0],4*20))
        ind_weight=torch.zeros(bbox_targets.size(),dtype=torch.float64)
        
        indexs=torch.where(labels>0)[0]
       # print(indexs)
        for index in indexs:
            cls=labels[index]
        
            
            strat=int(4*cls)
            end=strat+4
            #print(ind_weight[index,strat:end])
            bbox_targets[index,strat:end]=box_target[index]
            ind_weight[index,strat:end]=torch.tensor([1,1,1,1],dtype=torch.float64)
        return bbox_targets,ind_weight
    def for_tarining(self,gt_box):
        proposals,scores,keep=self.get_rois()
        
        all_ious=self.get_iou(proposals,gt_box)#(N,K)
        
        N_index=all_ious.argmax(axis=1)
        N_iou=all_ious[torch.arange(len(N_index)),N_index].reshape(-1,1).squeeze()
        
        Index_fg=torch.where(N_iou>0.5)
        Index_bg=torch.where(N_iou<0.5)
        
      
        indexs=torch.hstack((Index_fg[0],Index_bg[0]))
        labels=gt_box[N_index,-1]
        labels[len(Index_fg[0]):]=0
   
        rois=proposals[indexs,:]
       
     
       
        bbox_target=self.get_targets(rois,gt_box[N_index])
       
        
        #出现问题
        bbox_target,ind_weight=self.get_final_targets(labels,bbox_target)

        out_weight=torch.tensor(ind_weight>0,dtype=torch.float64)
       # print(torch.where(rois==torch.tensor([0,0,0,0]),device='cpu'))
        
        
        
        return rois,labels,bbox_target,ind_weight,out_weight
        


In [25]:
#train_rcnn(1000,images,dataset)

In [26]:
class faster_rcnn(nn.Module):
    def __init__(self):
        super(faster_rcnn,self).__init__()
        
        self.sub_layer1=nn.Conv2d(512,512,kernel_size=3,padding=1,stride=1)
        self.sub_relu=nn.ReLU()
        
        self.layer1_1=nn.Conv2d(512,18,kernel_size=3,padding=1,stride=1)
        self.layer1_2=nn.Conv2d(512,36,kernel_size=3,padding=1,stride=1)
         
        self.linear1=nn.Linear(8192,1000)
        self.relu1=nn.ReLU()
    
        self.linear2=nn.Linear(1000,100)
        self.relu2=nn.ReLU()
        
        self.linear3=nn.Linear(100,4*20)
        
        self.softmax=nn.Softmax(dim=1)
        
        self.linear4=nn.Linear(100,20)
        
        
    def reshape_layer(self,rpn_cls,dim):
        
        
        reshaped=rpn_cls.reshape((1,dim,-1,rpn_cls.size()[-1]))
        return reshaped
    
    def get_roi_feature_maps(self,feature_maps,rois,scale,width,height):
        
        rois=torch.round(rois/scale)
        n,c,w,h=feature_maps.size()
        
        length=rois.size()[0]
        maps=[]
        for i in range(length):
          
            h=int(rois[i,3].detach())-int(rois[i,1].detach())
            w=int(rois[i,2].detach())-int(rois[i,0].detach())
            
            
            if h<=0 :
                h=1
            if w<=0:
                w=1
           
            if rois[i,1]>=height:
                rois[i,1]=height-1
                h=1
            if rois[i,0]>=width:
                rois[i,0]=width-1
                w=1
           
            
            feature_map=feature_maps[:,:,int(torch.abs(rois[i,1].detach())):int(torch.abs(rois[i,1].detach())+h),int(torch.abs(rois[i,0].detach())):int(torch.abs(rois[i,0].detach())+w)]
           
            feature_map=Resize([16,16])(feature_map)
            
            
            feature_map=nn.MaxPool2d(kernel_size=(4,4),stride=4,padding=0,dilation=1,ceil_mode=False)(feature_map)
            
            feature_map=feature_map.reshape(1,-1)
            
            maps.append(feature_map)
            
        final=torch.vstack((maps))
        return final
        
   #命名有点问题！！！！ 
    def bbox_and_targets(self,anchors,rpn_bbox_pred):
        
        ws=anchors[:,2]-anchors[:,0]+1
        hs=anchors[:,3]-anchors[:,1]+1
        center_x=(anchors[:,0]+anchors[:,2])/2
        center_y=(anchors[:,1]+anchors[:,3])/2
        
        pred_x=rpn_bbox_pred[:,0]*ws+center_x
        pred_y=rpn_bbox_pred[:,1]*hs+center_y
        pred_w=torch.exp(rpn_bbox_pred[:,2])*ws
        pred_h=torch.exp(rpn_bbox_pred[:,3])*hs
        
        p_box_0=torch.round(pred_x-(pred_w-1)/2).reshape((-1,1))
        p_box_1=torch.round(pred_y-(pred_h-1)/2).reshape((-1,1))
        p_box_2=torch.round(pred_x+(pred_w-1)/2).reshape((-1,1))
        p_box_3=torch.round(pred_y+(pred_h-1)/2).reshape((-1,1))
        
        p_bbox=torch.hstack((p_box_0,p_box_1,p_box_2,p_box_3))
        return p_bbox
        
        
        
    def forward(self,x,gt_box):
        
        image=x
        x=vgg16(x)
        height,width=x.size()[2:]
        
        x=self.sub_relu(self.sub_layer1(x))
        
        rpn_cls=self.layer1_1(x)#(1,18,60,40)
        rpn_bbox=self.layer1_2(x)#(1,36,60,40)
        
        rpn_cls_reshaped=self.reshape_layer(rpn_cls,2)
        rpn_cls_pred=nn.Softmax(dim=1)(rpn_cls)
        rpn_cls=self.reshape_layer(rpn_cls_pred,18)##(1,18,60,40)
    
    
        anchors=get_base_anchors(width,height,16)()   
        #anchors=anchors.to('cuda:0')
        rpn_labels,rpn_box_target,rpn_ind_weight,rpn_out_weight=get_lots_on_base_anchors(rpn_cls,gt_box,anchors,16).get_lots_im()
        rois,labels,bbox_target,ind_weight,out_weight=get_proposal(rpn_cls,rpn_bbox,anchors,0.5,3000,20,image).for_tarining(gt_box)
        #print(rois.size())
        
        final=self.get_roi_feature_maps(x,rois,16,width,height)#(N,X)
        N,X=final.size()
        final=self.linear1(final)
        final=self.relu1(final)
        final=self.linear2(final)
        final=self.relu2(final)
        
        target_pred=self.linear3(final)
        
        cls_prod=self.softmax(self.linear4(final))
        
        final_labels=cls_prod.argmax(axis=1)
       
        tar=torch.hstack(((target_pred[torch.arange(len(final_labels)),4*final_labels].reshape((-1,1))),
                        (target_pred[torch.arange(len(final_labels)),4*final_labels+1].reshape((-1,1))),
                        (target_pred[torch.arange(len(final_labels)),4*final_labels+2].reshape((-1,1))),
                        (target_pred[torch.arange(len(final_labels)),4*final_labels+3].reshape((-1,1)))))
        
        box=self.bbox_and_targets(rois,tar)
        
        
        return box,final_labels,rpn_cls,rpn_bbox,rpn_labels,rpn_box_target,rpn_ind_weight,rpn_out_weight,cls_prod,target_pred,labels,bbox_target,ind_weight,out_weight


In [27]:
net=faster_rcnn()
optimizer = optim.Adam(net.parameters(), lr=0.00001) #设置优化器和学习率

In [28]:
class rcnn_loss(nn.Module):
    def __init__(self,cls_prod,target_pred,labels,bbox_target,ind_weight,out_weight,
                rpn_scores,rpn_box_pred,rpn_labels,rpn_box_target,rpn_ind_weight,rpn_out_weight):
        super(rcnn_loss,self).__init__()
        #final
        self.cls_prod=cls_prod
        self.target_pred=target_pred
        self.labels=labels
        self.bbox_target=bbox_target
        self.ind_weight=ind_weight
        self.out_weight=out_weight
        #rpn
        self.rpn_scores=rpn_scores
        self.rpn_box_pred=rpn_box_pred
        self.rpn_labels=rpn_labels
        self.rpn_box_target=rpn_box_target
        self.rpn_ind_weight=rpn_ind_weight
        self.rpn_out_weight=rpn_out_weight
   
    def l1_smooth(self,target_pred,bbox_target,ind_weight,out_weight):
   
        
        box_diff=target_pred-bbox_target
        
        in_box_diff=ind_weight*box_diff
        abs_in_box_diff=torch.abs(in_box_diff)
        
        index=abs_in_box_diff<1
        L1_loss=torch.pow(abs_in_box_diff,2)*index+(-1*index+1)*(abs_in_box_diff-0.5)
        
        L1_loss=L1_loss*out_weight
        L1_loss=torch.mean(L1_loss)
        
        return L1_loss
    
    def labels_loss_for_rpn(self,rpn_labels,rpn_scores):
        rpn_labels=rpn_labels
        rpn_scores=rpn_scores
        rpn_scores=rpn_scores.reshape(-1,2)
        rpn_select=torch.where(torch.not_equal(rpn_labels,-1))[0]
        rpn_scores=rpn_scores[rpn_select,:]
        rpn_labels=rpn_labels[rpn_select]
        rpn_scores=nn.Softmax(dim=1)(rpn_scores)
        rpn_scores_index=rpn_scores.argmax(axis=1)
        
        loss=nn.BCEWithLogitsLoss()(rpn_scores[torch.arange(len(rpn_scores_index)),rpn_scores_index],rpn_labels)
        
        loss=torch.mean(loss)
        return loss
    def labels_loss_for_final(self,cls_prod,labels):#(N,20)
        
        cls_prod=cls_prod
        labels=labels
        cls_prod_index=cls_prod.argmax(axis=1)
        
        loss=nn.BCEWithLogitsLoss()(cls_prod[torch.arange(len(cls_prod_index)),cls_prod_index],labels.float())
        loss=torch.mean(loss)
        
        return loss
    def forward(self):
        
       
        rpn_loss=self.l1_smooth(self.rpn_box_pred,self.rpn_box_target,self.rpn_ind_weight,
                                self.rpn_out_weight)+self.labels_loss_for_rpn(self.rpn_labels,self.rpn_scores)
        
        final_loss=self.l1_smooth(self.target_pred,self.bbox_target,self.ind_weight,
                                  self.out_weight)+self.labels_loss_for_final(self.cls_prod,self.labels)
        
        all_loss=rpn_loss+final_loss
        
        
        return all_loss
   

In [29]:
def draw_one(image,xmin,ymin,xmax,ymax,str,font,color='black',thickness=4):
    draw = ImageDraw.Draw(image)
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
    draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
    text_bottom = bottom
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle(
      [(left, text_bottom - text_height - 2 * margin), (left + text_width,
                                                        text_bottom)],
      fill=color)
    draw.text(
      (left + margin, text_bottom - text_height - margin),
      display_str,
      fill='black',
      font=font)

    return image
    

In [30]:
def draw_it(image,labels,box):
    
    num_box=box.size()[0]
    
    image=image.numpy()
    
    for i in range(num_box):
        the_class=labels[i]
        xmin,ymin,xmax,ymax=box[i,0],box[i,1],box[i,2],box[i,3]
        image=draw_one(image,xmin,ymin,xmax,ymax,'%s'%(the_class),FONT)
    image=torch.tensor(image)  
    return image    

In [33]:
def train_rcnn(epoches,images,dataset):
    
    length=len(dataset)
    net.train()
    for i in range(epoches):
        num=0
        print(num)

        for j in range(len(images)):
            optimizer.zero_grad()
            image=images[j]
            image=image.unsqueeze(0)
            gt_box=torch.tensor(dataset[j])
          
            
       #     image = image.to(device)
       #     gt_box = gt_box.to(device)
            num+=1
            box,final_labels,rpn_cls,rpn_bbox,rpn_labels,rpn_box_target,rpn_ind_weight,rpn_out_weight,cls_prod,target_pred,labels,bbox_target,ind_weight,out_weight=net(
            image,gt_box)
      
            
            loss=rcnn_loss(cls_prod,target_pred,labels,bbox_target,ind_weight,out_weight,rpn_cls,rpn_bbox,
                           rpn_labels,rpn_box_target,rpn_ind_weight,rpn_out_weight)()
            loss.backward()
            optimizer.step()
            print('a'*100)
            
            
            if num%2==0:
                
                index1=cls_prod.argmax(axis=1)
                box_pred2=torch.zeros_like(box_pred[:,0:4])
                for i in range(len(box_pred2)):
                    box_pred2[i]=box_pred[i,index1[i]:index1[i]+4]
                cls_prod=cls_prod[torch.arange(len(index1)),index1]
                index2=torch.where(cls_prod>0.8)[0]
                cls_prod=torch.round(cls_prod[index2]).type(torch.int)
                if len(index2)==0:
                    break
                else:
                    boxes=tools.change_on_pre(rois,box_pred2)[index2,:]
                    boxes=torch.round(boxes).type(torch.int)
                    list_name=[]
                    for i in range(len(cls_prod)):
                        list_name.append(name[cls_prod[i]])
                    image=draw_it(image,list_name,boxes)
                    image=torch.squeeze(image.to('cpu'))
                    plt.imshow(image)
   

In [34]:
train_rcnn(100,images,dataset)

0


<ipython-input-22-e6d8cfc7c49d>:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale=torch.tensor(self.scale)
<ipython-input-24-70da47ee8c2f>:219: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out_weight=torch.tensor(ind_weight>0,dtype=torch.float64)


aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
0


ValueError: cannot convert float NaN to integer